In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix
from pyomo.opt import SolverFactory
import pandas as pd

In [2]:
load = pd.read_csv("load.tsv", delim_whitespace=True, names=["time", "value"] )
load

,time,value
0,t1,82115
1,t2,73169
2,t3,68729
3,t4,63442
4,t5,60430
5,t6,57013
6,t7,52048
7,t8,48701
8,t9,43981
9,t10,40498


In [21]:
duration = pd.read_csv("duration.tsv", delim_whitespace=True, names=["time", "hour"] )
duration

,time,hour
0,t1,102
1,t2,962
2,t3,962
3,t4,962
4,t5,962
5,t6,962
6,t7,962
7,t8,962
8,t9,962
9,t10,962


In [35]:
availability = pd.read_csv("availability.tsv", sep="\s+", header=0, index_col=False  )
availability

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629
1,GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980
2,Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012
3,Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564
4,Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687
5,Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610
6,Wind,11480,9858,11332,14498,20590,21012,6314,16072,21238
7,Solar,10538,2078,3920,7641,6970,0,14810,1307,0


In [3]:
tech_data = pd.read_csv("tech_data.tsv", sep="\s+", header=0, index_col=False ) 
tech_data.set_index("tech", inplace=True)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF
tech,,,,,
*Units,MW,-,Euro/MWhtherm,Euro/Mwhel,t
CCGT,29555,0.54,12.8,1.5,0.2048
GT_GasOil,4400,0.28,12.8,1.5,0.2048
Hydro,5256,1.00,0.0,1.5,0.0000
Coal,22458,0.42,7.4,2.6,0.3420
Lignite,21067,0.37,3.4,3.0,0.3996
Nuclear,8114,0.33,1.8,0.7,0.0000
Wind,61114,1.00,0.0,1.4,0.0000
Solar,46471,1.00,0.0,1.0,0.0000


In [9]:
gens = ["CCGT","GT_GasOil","Hydro","Coal","Lignite","Nuclear","Wind","Solar"]

data = pd.DataFrame(index=gens)

data["Fuel_P"] = [12.8,12.8,0.0,7.4,3.4,1.8,0.0,0.0]
data["Cap"] = [29555,4400,5256,22458,21067,8114,61114,46471]
data["ETA_EL"] = [0.54,0.28,1.00,0.42,0.37,0.33,1.00,1.00]
data["EMF"] = [0.2048,0.2048,0.00,0.3420,0.3996,0.00,0.00,0.00]
data["c_var_other"] = [1.5,1.5,1.5,2.6,3.0,0.7,1.4,1.0]

data

,Fuel_P,Cap,ETA_EL,EMF,c_var_other
CCGT,12.8,29555,0.54,0.2048,1.5
GT_GasOil,12.8,4400,0.28,0.2048,1.5
Hydro,0.0,5256,1.00,0.0000,1.5
Coal,7.4,22458,0.42,0.3420,2.6
Lignite,3.4,21067,0.37,0.3996,3.0
Nuclear,1.8,8114,0.33,0.0000,0.7
Wind,0.0,61114,1.00,0.0000,1.4
Solar,0.0,46471,1.00,0.0000,1.0


In [13]:
data["costs_elwithoutCO2"] = data.Fuel_P/data.ETA_EL+data.c_var_other
data.costs_elwithoutCO2

CCGT         25.203704
GT_GasOil    47.214286
Hydro         1.500000
Coal         20.219048
Lignite      12.189189
Nuclear       6.154545
Wind          1.400000
Solar         1.000000
Name: costs_elwithoutCO2, dtype: float64

In [41]:
gens = ["CCGT","GT_GasOil","Hydro","Coal","Lignite","Nuclear","Wind","Solar"]

data1= pd.DataFrame(index=gens)

data1["t1"] = [19629,2980,4012,19564,17687,7610,11480,10538]
data1["t2"] = [19629,2980,4012,19564,17687,7610,9859,2078]
data1["t3"] = [19629,2980,4012,19564,17687,7610,11332,3920]
data1["t4"] = [19629,2980,4012,19564,17687,7610,14498,7641]
data1["t5"] = [19629,2980,4012,19564,17687,7610,20590,6970]
data1["t6"] = [19629,2980,4012,19564,17687,7610,21012,0]
data1["t7"] = [19629,2980,4012,19564,17687,7610,6314,14810]
data1["t8"] = [19629,2980,4012,19564,17687,7610,16072,1307]
data1["t9"] = [19629,2980,4012,19564,17687,7610,21238,0]
data1["t10"] = [19629,2980,4012,19564,17687,7610,12054,0]

data1

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629,19629
GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980,2980
Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564,19564
Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687,17687
Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610,7610
Wind,11480,9859,11332,14498,20590,21012,6314,16072,21238,12054
Solar,10538,2078,3920,7641,6970,0,14810,1307,0,0


In [42]:
data["costs_elwithoutCO2time"] = expr = sum([data.costs_elwithoutCO2[gen]*model.x[gen] for gen in gens]
data.costs_elwithoutCO2time

AttributeError: 'DataFrame' object has no attribute 'costs_elwithoutCO2'

In [15]:
data["costs_elwithCO2"] = data.Fuel_P/data.ETA_EL+data.c_var_other+data.EMF*50/data.ETA_EL
data.costs_elwithCO2

CCGT         44.166667
GT_GasOil    83.785714
Hydro         1.500000
Coal         60.933333
Lignite      66.189189
Nuclear       6.154545
Wind          1.400000
Solar         1.000000
Name: costs_elwithCO2, dtype: float64

In [17]:
data["STMGC_withoutCO2"]=data.Fuel_P/data.ETA_EL
data.STMGC_withoutCO2

CCGT         23.703704
GT_GasOil    45.714286
Hydro         0.000000
Coal         17.619048
Lignite       9.189189
Nuclear       5.454545
Wind          0.000000
Solar         0.000000
Name: STMGC_withoutCO2, dtype: float64

In [20]:
data["STMGC_withCO2"]=data.Fuel_P/data.ETA_EL+data.EMF*50/data.ETA_EL
data.STMGC_withCO2

CCGT         42.666667
GT_GasOil    82.285714
Hydro         0.000000
Coal         58.333333
Lignite      63.189189
Nuclear       5.454545
Wind          0.000000
Solar         0.000000
Name: STMGC_withCO2, dtype: float64

In [23]:
demand = 500
co2_limit = 250
#co2_limit = 300

In [24]:
model = ConcreteModel()

model.x = Var(gens, domain=NonNegativeReals)

model.balance = Constraint(expr = model.x["gas"] + model.x["coal"] == demand)

def cap_limits(model,gen):
    return model.x[gen] <= data.capacities[gen]

model.cap_limits = Constraint(gens,rule=cap_limits)

model.emissions = Constraint(expr = sum([data.emissions_el[gen]*model.x[gen] for gen in gens]) <= co2_limit)

model.objective = Objective(expr = sum([data.costs_el[gen]*model.x[gen] for gen in gens]))

In [25]:
opt = SolverFactory("glpk")

In [26]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [27]:
results = opt.solve(model,suffixes=["dual"])

In [28]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14000.0
  Upper bound: 14000.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.010906696319580078
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [29]:
for gen in gens:
    print(gen,model.x[gen].value)

gas 300.0
coal 200.0


In [30]:
model.dual[model.balance]

44.0

In [31]:
model.dual[model.emissions]

-32.0